In [3]:
import cv2
import numpy as np
from pathlib import Path
from PIL import Image
import gradio as gr


class MosaicMaker:
    def __init__(self, tile_dir: str, cell_size: int = 32):
        """
        - tile_dir: directory containing tile images
        - cell_size: the pixel size of a single tile (must be int), e.g. 32/16/8
        """
        assert isinstance(cell_size, int) and cell_size > 0
        self.cell_size = cell_size
        self.tiles, self.tile_colors = self._load_tiles(tile_dir)
    

    # Step 1: Image Selection and Preprocessing
    def _load_tiles(self, tile_dir: str):
        """
        Load tile images from the specified directory, resize them to (cell_size, cell_size),
        and compute their average colors.
        """
        tiles = []
        tile_colors = []
        valid_exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp"}

        for path in Path(tile_dir).rglob("*"):
            if path.suffix.lower() not in valid_exts:
                continue
            tile_bgr = cv2.imread(str(path))
            if tile_bgr is None:
                continue
            tile = cv2.cvtColor(tile_bgr, cv2.COLOR_BGR2RGB)
            # Here, resize_with_crop needs a (width, height) tuple
            tile = self.resize_with_crop(tile, (self.cell_size, self.cell_size))
            tiles.append(tile)
            tile_colors.append(tile.mean(axis=(0, 1)))

        if not tiles:
            raise RuntimeError(f"No valid tiles found in: {tile_dir}")

        return tiles, np.asarray(tile_colors, dtype=np.float32)

    # Step 2: Image Griding and Thresholding
    def color_quantization(self, img_rgb: np.ndarray, k: int = 16) -> np.ndarray:
        """
        Perform color quantization using K-means and return the quantized image (keep RGB).
        """
        Z = img_rgb.reshape((-1, 3)).astype(np.float32)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
        _compactness, labels, centers = cv2.kmeans(
            Z, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS
        )
        centers = centers.astype(np.uint8)
        res = centers[labels.flatten()]
        return res.reshape(img_rgb.shape)


    def resize_with_crop(self, img_rgb: np.ndarray, target_size=(512, 512)) -> np.ndarray:
        """
        Resize the image to cover the target_size while maintaining aspect ratio,
        then center-crop to the fixed resolution.
        """
        th, tw = target_size
        h, w = img_rgb.shape[:2]
        # Determine scale to cover target size
        scale = max(th / h, tw / w)
        new_w = max(1, int(round(w * scale)))
        new_h = max(1, int(round(h * scale)))
        resized = cv2.resize(img_rgb, (new_w, new_h), interpolation=cv2.INTER_AREA)

        x0 = max(0, (new_w - tw) // 2)
        y0 = max(0, (new_h - th) // 2)
        cropped = resized[y0:y0 + th, x0:x0 + tw]
        return cropped

    
    def grid_mean_colors(self, img_rgb: np.ndarray) -> np.ndarray:
        """
        Divide the image into a grid of cells (cell_size x cell_size),
        compute the mean color of each cell, and fill the cell with this mean color.
        Input: (H, W, 3), where H and W must be divisible by cell_size
        Output: (H, W, 3), each cell filled with its mean color
        """
        H, W = img_rgb.shape[:2]
        cs = self.cell_size
        if (H % cs != 0) or (W % cs != 0):
            raise ValueError(
                f"Image size {H}x{W} must be divisible by cell_size={cs} "
            )
    
        out = np.zeros_like(img_rgb, dtype=np.uint8)
    
        for i in range(0, H, cs):
            for j in range(0, W, cs):
                cell = img_rgb[i:i+cs, j:j+cs]
                mean_color = cell.mean(axis=(0, 1))
                out[i:i+cs, j:j+cs] = mean_color  # Broadcasting
    
        return out


    # Step 3: Tile Mapping
    def build_mosaic(self, mean_color_img: np.ndarray) -> np.ndarray:
        """
        Build the mosaic image by mapping each cell's mean color to the closest tile color.
        """
        H, W = mean_color_img.shape[:2]
        cs = self.cell_size
        n_rows, n_cols = H // cs, W // cs
    
        out = np.zeros((H, W, 3), dtype=np.uint8)
    
        for i in range(n_rows):
            for j in range(n_cols):
                y1, y2 = i * cs, (i + 1) * cs
                x1, x2 = j * cs, (j + 1) * cs
    
                mean_color = mean_color_img[y1:y2, x1:x2].mean(axis=(0, 1))
                dists = np.linalg.norm(self.tile_colors - mean_color, axis=1)
                idx = int(np.argmin(dists))
                out[y1:y2, x1:x2] = self.tiles[idx]
    
        return out


    def blend_mosaic_with_ref(self, mosaic_img: np.ndarray, ref_img: np.ndarray, alpha: float = 0.5) -> np.ndarray:
        """
        Blend the mosaic and reference image cell by cell. Both must have the same shape.
        """
        if mosaic_img.shape != ref_img.shape:
            raise ValueError("mosaic_img and ref_img must have the same shape.")

        H, W, _ = mosaic_img.shape
        cs = self.cell_size
        a = float(alpha)
        if a <= 0:
            return mosaic_img.copy()
        if a >= 1:
            return ref_img.copy()

        out = mosaic_img.copy()
        for y in range(0, H, cs):
            for x in range(0, W, cs):
                tile = out[y:y + cs, x:x + cs].astype(np.float32)
                cell = ref_img[y:y + cs, x:x + cs].astype(np.float32)
                blended = (1.0 - a) * tile + a * cell
                out[y:y + cs, x:x + cs] = np.clip(blended, 0, 255).astype(np.uint8)
        return out


# Step 4: Building the Gradio Interface
with gr.Blocks(title="Image Tile Mosaic") as demo:
    gr.Markdown("## Upload an image and generate a mosaic.")
    with gr.Row():
        inp_img = gr.Image(type="pil", label="Input Image")
        out_img = gr.Image(type="pil", label="Output Mosaic")
    with gr.Row():
        cell = gr.Radio(choices=[8, 16, 32], value=32, label="Cell Size (px)")
        blend = gr.Slider(0.0, 0.8, value=0.0, step=0.05, label="Blend with original (per cell)")

    btn = gr.Button("Generate")

    def _run(pil_img: Image.Image, cell: int, blend: float):
        # 1) PIL to NumPy
        img = np.array(pil_img.convert("RGB"))

        # 2) Resize to a fixed size (recommended 512x512 for easy division)
        target = (512, 512)
        maker = MosaicMaker("tiles_folder", cell_size=int(cell))
        img_512 = maker.resize_with_crop(img, target)

        # Check divisibility (if slider gives a non-divisible cell_size, an error will be raised here)
        H, W = img_512.shape[:2]
        if (H % maker.cell_size != 0) or (W % maker.cell_size != 0):
            raise gr.Error(f"Target size {H}x{W} is not divisible by cell={maker.cell_size}. "
                           f"Try cell sizes that divide 512, e.g. 32, 16, 8.")

        # 3) Color Quantization
        quant = maker.color_quantization(img_512, k=16)

        mean_colors = maker.grid_mean_colors(quant)

        # 4) Generate Mosaic
        mosaic = maker.build_mosaic(mean_colors)

        # 5) Blend with original if needed
        final = maker.blend_mosaic_with_ref(mosaic, mean_colors, alpha=float(blend))

        return Image.fromarray(final)

    btn.click(_run, inputs=[inp_img, cell, blend], outputs=[out_img])
    demo.launch()



* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
